In [260]:
import warnings

import geopandas as gpd
import numpy as np
import pandas as pd
import seaborn as sns
from shapely.geometry import Point

# Pandas settings
pd.options.display.max_columns = None
pd.options.display.max_rows = 200
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Seaborn settings
sns.set_style('whitegrid')

# Warnings
warnings.filterwarnings('ignore')

In [261]:
# Read Grid (1km x 1km)
path = r"./data/GRID_NSP2021_RES/GRID_NSP2021_RES.shp"
gdf = gpd.read_file(path)

In [ ]:
# Show Grid
# We'll be using 'RES' column
# 'geometry' column stores shape
gdf

In [ ]:
# Show Grid sample
gdf.sample(3000).explore()

In [ ]:
# Show Grid head
gdf.head(3000).explore()

In [ ]:
# Read Gminy
path = r"./data/Gminy/Gminy.shp"
gminy = gpd.read_file(path)

display(gminy.shape)
display(gminy.head(1))

In [ ]:
# Filter Gminy for 'Warszawa'
gminy_waw = gminy.loc[gminy['JPT_NAZWA_'] == 'Warszawa']
gminy_waw.explore()

In [ ]:
print("Gminy CRS: ")
display(gminy.crs)
print("\nGrid CRS:")
display(gdf.crs)

In [268]:
# Convert CRS to degree
gdf = gdf.to_crs('EPSG:4258')

In [ ]:
# Show Grid in Warsaw
gdf.sjoin(gminy_waw).explore()

In [ ]:
# Show Grid in Warsaw
gdf.overlay(gminy_waw).explore()

In [271]:
# Use sjoin
gdf_waw = gdf.sjoin(gminy_waw)
gdf_waw = gdf_waw.drop('index_right', axis=1)

In [272]:
# Raed POS Database
pos_db = pd.read_excel("./data/POS_DB/POS.xlsx")

# Preprocessing
pos_db.isna().sum()

idx = pos_db.loc[pos_db[['Longitude', 'Latitude']].isna().any(axis=1)].index
pos_db = pos_db.drop(idx).reset_index(drop=True)

pos_db['Latitude'] = pos_db['Latitude'].astype(str).str.replace(",", "")
pos_db['Longitude'] = pos_db['Longitude'].astype(str).str.replace(",", "")

pos_db['Latitude'] = pos_db['Latitude'].astype(float)
pos_db['Longitude'] = pos_db['Longitude'].astype(float)

In [273]:
# Convert DataFrame to GeoDataFrame (set up CRS)
pos_db['geometry'] = pos_db.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)
pos_db = gpd.GeoDataFrame(pos_db, crs=4258)

In [ ]:
# Show sample POS
pos_db.sample(2000).explore()

In [ ]:
# Show stores in Warsaw
pos_db.sjoin(gdf_waw).explore()

In [276]:
# Add ID to merge Number of Stores in Grid
gdf_waw = gdf_waw.reset_index(drop=True)
gdf_waw['id'] = gdf_waw.index

# Sjoin stores to Grid in Warsaw and aggregate to get number of stores in each box
stores_agg = gdf_waw.sjoin(pos_db, how='left')
stores_agg = stores_agg.groupby('id', as_index=False).agg(Num_Stores=('Company', 'count'))

# Add Number of Stores to Grid
gdf_waw = gdf_waw.merge(stores_agg, on='id', how='left', validate='1:1')

# Add Number of Stores per Person
gdf_waw['Num_Stores_per_1k_ppl'] = (gdf_waw['Num_Stores'] / (gdf_waw['RES'] / 1000)).fillna(0).replace(np.inf, 0).replace(-np.inf, 0)

In [ ]:
gdf_waw['Num_Stores'].value_counts()

In [ ]:
# Show areas where number of stores is 0
gdf_waw.loc[gdf_waw['Num_Stores'] == 0].explore()

In [279]:
# Score
gdf_waw['Score'] = 1 - gdf_waw['Num_Stores_per_1k_ppl']

In [ ]:
gdf_waw.explore(
    style_kwds={
        "style_function": lambda x: {"fillOpacity": x["properties"]["Score"] * 0.8}
    }
)